In [2]:
import lightgbm as lgb
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
import lightgbm as lgb
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
import joblib
import warnings

warnings.filterwarnings('ignore')

# ===============================
# 1️⃣ Data Loading & Preprocessing
# ===============================

# Load the dataset
filename = "/Users/arthur/Documents/GITHUB/Rough_informer/Rough_Informer_for_High_freq_Order_Book/final_dataset_depth-2_row_1000.csv"
df = pd.read_csv(filename)

# Separate features and target
X = df.drop(columns=['y', 'time_step']).values  # Drop 'time_step' as it's metadata
y = df['y'].values

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Replace train_test_split with:
split_idx = int(len(df) * 0.8)
X_train, X_val = X_scaled[:split_idx], X_scaled[split_idx:]
y_train, y_val = y[:split_idx], y[split_idx:]

In [2]:
print("X_train", X_train.shape)
print("X_val", X_val.shape)
print("y_train", y_train.shape)
print("y_val", y_val.shape)

X_train (800, 1260)
X_val (200, 1260)
y_train (800,)
y_val (200,)


In [3]:

# ===============================
# 2️⃣ Hyperparameter Tuning with Hyperopt
# ===============================

param_space = {
    'boosting_type': hp.choice('boosting_type', ['dart']),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 0.6),
    'drop_rate': hp.uniform('drop_rate', 0.4, 0.9),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.3)),
    'max_depth': hp.choice('max_depth', [3, 4, 5, 6]),
    'min_child_samples': hp.randint('min_child_samples', 20, 30),
    'min_child_weight': hp.uniform('min_child_weight', 0.01, 0.05),
    'n_estimators': hp.choice('n_estimators', [100, 200, 300]),
    'objective': 'regression',
    'skip_drop': hp.uniform('skip_drop', 0.5, 0.9),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'reg_alpha': hp.uniform('reg_alpha', 0, 1),
    'reg_lambda': hp.uniform('reg_lambda', 0, 1),
}

def train_lgb_model(params, X_train, y_train, X_val, y_val, seed=42):
    """Train LightGBM and return (model, R^2, RMSE) on validation."""
    model = lgb.LGBMRegressor(
        boosting_type=params['boosting_type'],
        colsample_bytree=params['colsample_bytree'],
        drop_rate=params['drop_rate'],
        learning_rate=params['learning_rate'],
        max_depth=params['max_depth'],
        min_child_samples=params['min_child_samples'],
        min_child_weight=params['min_child_weight'],
        n_estimators=params['n_estimators'],
        objective=params['objective'],
        skip_drop=params['skip_drop'],
        subsample=params['subsample'],
        random_state=seed,
        n_jobs=-2,
        verbosity=-1
        reg_alpha=params['reg_lambda']
        reg_lambda=params['reg_lambda'])
    
    
    # Correct way to pass early stopping
    model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric='l2',  # Use MSE for early stopping
    callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)])
    
    y_val_pred = model.predict(X_val)
    r2_val = r2_score(y_val, y_val_pred)
    rmse_val = np.sqrt(mean_squared_error(y_val, y_val_pred))
    
    return model, r2_val, rmse_val


def objective_func(space):
    _, r2_val, rmse_val = train_lgb_model(space, X_train, y_train, X_val, y_val)
    return {'loss': 1.0 - r2_val, 'status': STATUS_OK}

# Hyperparameter Optimization
trials = Trials()
best_params = fmin(
    fn=objective_func,
    space=param_space,
    algo=tpe.suggest,
    max_evals=5,
    trials=trials,
    rstate=np.random.default_rng(42)
)

SyntaxError: invalid syntax (<ipython-input-3-b0f225a7dae2>, line 38)

In [33]:
print(best_params)

{'boosting_type': 0, 'colsample_bytree': 0.6795661544916977, 'drop_rate': 1, 'drop_rate_conditional': 0.19396143153138945, 'learning_rate': 0.25474246690912433, 'max_depth': 0, 'min_child_samples': 161, 'n_estimators': 0, 'skip_drop': 0, 'subsample': 0.9268488838215319}


In [17]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
import lightgbm as lgb
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
import joblib
import warnings

warnings.filterwarnings('ignore')

# ===============================
# 1️⃣ Data Loading & Preprocessing
# ===============================

# Load the dataset
filename = "/Users/arthur/Documents/GITHUB/Rough_informer/Rough_Informer_for_High_freq_Order_Book/final_dataset_depth-2_row_10_000.csv"
df = pd.read_csv(filename)

# Separate features and target
X = df.drop(columns=['y', 'time_step']).values  # Drop 'time_step' as it's metadata
y = df['y'].values

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Replace train_test_split with:
split_idx = int(len(df) * 0.8)
X_train, X_val = X_scaled[:split_idx], X_scaled[split_idx:]
y_train, y_val = y[:split_idx], y[split_idx:]
print(X.shape)


(10000, 1260)


In [18]:
print("X_train", X_train.shape)
print("X_val", X_val.shape)
print("y_train", y_train.shape)
print("y_val", y_val.shape)

X_train (8000, 1260)
X_val (2000, 1260)
y_train (8000,)
y_val (2000,)


In [8]:


# ===============================
# 3️⃣ Final Model Training
# ===============================



# Train the final model
final_model, final_r2, final_rmse = train_lgb_model(best_params, X_train, y_train, X_val, y_val)

# Save the model
joblib.dump(final_model, 'lgb_model_final.joblib')

# ===============================
# 4️⃣ Evaluation
# ===============================

print(f"Final R² on Validation Set: {final_r2:.4f}")
print(f"Final RMSE on Validation Set: {final_rmse:.4f}")


[1]	valid_0's l2: 0.477084
[2]	valid_0's l2: 0.477084
[3]	valid_0's l2: 0.477084
[4]	valid_0's l2: 0.477084
[5]	valid_0's l2: 0.477084
[6]	valid_0's l2: 0.477084
[7]	valid_0's l2: 0.477084
[8]	valid_0's l2: 0.477084
[9]	valid_0's l2: 0.477084
[10]	valid_0's l2: 0.477084
[11]	valid_0's l2: 0.477084
[12]	valid_0's l2: 0.477084
[13]	valid_0's l2: 0.477084
[14]	valid_0's l2: 0.477084
[15]	valid_0's l2: 0.477084
[16]	valid_0's l2: 0.477084
[17]	valid_0's l2: 0.477084
[18]	valid_0's l2: 0.477084
[19]	valid_0's l2: 0.477084
[20]	valid_0's l2: 0.477084
[21]	valid_0's l2: 0.477084
[22]	valid_0's l2: 0.477084
[23]	valid_0's l2: 0.477084
[24]	valid_0's l2: 0.477084
[25]	valid_0's l2: 0.477084
[26]	valid_0's l2: 0.477084
[27]	valid_0's l2: 0.477084
[28]	valid_0's l2: 0.477084
[29]	valid_0's l2: 0.477084
[30]	valid_0's l2: 0.477084
[31]	valid_0's l2: 0.477084
[32]	valid_0's l2: 0.477084
[33]	valid_0's l2: 0.477084
[34]	valid_0's l2: 0.477084
[35]	valid_0's l2: 0.477084
[36]	valid_0's l2: 0.477084
[